In [2]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

import numpy as np

# Helper Functions

In [25]:
def images_to_vectors(images, length):
    # reshapes the input iamge into a vector of given length
    # just checking that information isnt lost in the covnersion
    if np.prod(images.size()[1:]) > length:
        length = np.prod(images.size()[1:])
    return images.view(images.size(0), length)

def vectors_to_images(vectors, channel, width, height):
    # just error checking
    assert np.prod(images.size()[1:]) == channel*width*height
    return vectors.view(vectors.size(0), channel, width, height)

def noise(size, batchSize):
    n = Variable(torch.randn(size, batchSize))
    if torch.cuda.is_available(): return n.cuda() 
    return n

def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data

In [4]:
# unlike tensorflow where we have to define the graphs and their operations, in pytorch we deine the network as a class
# i.e we just define the operations and the forward prop, the backprop is sorta in built

# we get this done by defining our class that inherits fromthe torchnn.module and hence the other functions
# such as backprop can be used

In [10]:
# hyper parameters
bSize = 100
learning_rate = 0.0002

features = 784

# Discriminator

In [19]:
class LeDiscriminator(torch.nn.Module):
    def __init__(self, n_feat):
        super().__init__()
        # input vector
        n_features = n_feat
        # bianry output
        n_out = 1

        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

# Generator Network

In [22]:
class LeGenereator(torch.nn.Module):

    def __init__(self, batchSize, vecOut):
        super().__init__()
        # number of images to produce
        n_features = batchSize
        # length of vector that will be input to the Discriminator
        n_out = vecOut
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    

# Defining Network Essentials

In [23]:
discriminator = LeDiscriminator(features)
generator = LeGenereator(bSize, features)
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

In [24]:
# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)

# Loss function
loss = nn.BCELoss()

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1
# Number of epochs
num_epochs = 200